## <span style="font-family:Georgia, serif;">**Twitter Sentiment Analysis** :Understanding Emotions in Tweets about Apple and Google products.</span>

![alt text](twits.jpg "Title")

## Overview

## Business Understanding

**Business Problem**: Using Sentiment Analysis to Improve Apple and Google Product Marketing Strategies 

The introduction of social media has completely changed how businesses interact with their consumers and the general public in today's connected society. While the digital age offers limitless possibilities for marketing and brand development, it also brings its own set of difficulties. One of these difficulties is the inability of enterprises to precisely gauge public opinion and feelings towards their goods or services.

In the age of social media, organizations are acutely aware of the need to harness the wealth of sentiment and emotion data available on these platforms. However, they often struggle to do so effectively, given the unprecedented speed, diversity, and complexity of social media communication. The dynamic nature of the medium, the diverse and contextual language used, the rapid increase of emojis and visual content, the volume of noise, and ethical concerns all contribute to the challenge of gauging public sentiment and emotions. 

To overcome these challenges, organizations must invest in advanced sentiment analysis tools and technologies, develop cultural and linguistic expertise, and strike a balance between data-driven insights and ethical considerations. By doing so, they can unlock the valuable insights hidden within the social media storm and use them to inform strategic decisions, enhance products and services, and build stronger connections with their audience in this rapidly evolving digital landscape.




## Data understanding

## Data Preparation

In [1]:
import pandas as pd
import re
import io
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

def remove_non_utf8(text):
    return re.sub(r'[^\x00-\x7F]+', '', text)

with open('data/judge_1377884607_tweet_product_company.csv', 'r', encoding='utf-8') as file:
    cleaned_text = remove_non_utf8(file.read())

df = pd.read_csv(io.StringIO(cleaned_text))
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [2]:
column_name_mapping = {'is_there_an_emotion_directed_at_a_brand_or_product': 'Sentiment'}
# Rename the columns using the .rename() method
df.rename(columns=column_name_mapping, inplace=True)


In [3]:
df['emotion_in_tweet_is_directed_at'].fillna('N/A', inplace=True)
df['tweet_text'].fillna('N/A', inplace=True)

In [4]:
def assign_brand(phrase):
    if 'iPad' in phrase or 'iPhone' in phrase :
        return 'Apple'
    elif 'Other Apple product or service' in phrase or 'Apple' in phrase:
        return 'Apple' 
    elif 'iPad or iPhone App' in phrase:
        return 'Apple'       
    elif 'Google' in phrase or 'Other Google product or service' in phrase:
        return 'Google'
    elif 'Android App' in phrase or 'Android' in phrase:
        return 'Android'
    else:
        return 'N/A'

df['brand'] = df['emotion_in_tweet_is_directed_at'].apply(assign_brand)

In [5]:
import nltk
import re
from nltk.tokenize import word_tokenize,TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# nltk.download('punkt')
# nltk.download('stopwords')


def clean_and_preprocess_text(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]
    # Remove mentions (words starting with '@') and URLs
    tokens = [token for token in tokens if not token.startswith('@') and not token.startswith('http')]
    # Remove punctuation and numbers using regular expressions
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # # Apply stemming using the Porter Stemmer
    # stemmer = PorterStemmer()
    # stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    cleaned_text = ' '.join(filtered_tokens) 
    return cleaned_text

In [6]:
df['tweet_text'][4]

"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)"

In [7]:
clean_and_preprocess_text(df['tweet_text'][4])

'great stuff fri sxsw  marissa mayer  google   tim oreilly  tech books  conferences   matt mullenweg  wordpress '

In [8]:
df['processed_text'] = df['tweet_text'].map(clean_and_preprocess_text)

In [9]:
# Define a mapping dictionary
sentiment_mapping = {'No emotion toward brand or product': 2.0,
                  'Positive emotion': 1.0, 
                  'Negative emotion': 0.0,
                  'I can\'t tell': 2.0}

# Use the .map() method to map values in column 'A' to new values
df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)
df.sample(5)

,tweet_text,emotion_in_tweet_is_directed_at,Sentiment,brand,processed_text
8308,wish i could go @mention Head over to {link} b...,N/A,2.0,N/A,wish could go head link pm cst today win vip...
424,Headed to Designing iPad Interfaces - New Navi...,N/A,2.0,N/A,headed designing ipad interfaces new navigati...
3124,"&quot;google has too many products, and needs ...",Google,0.0,Google,google many products needs condense sxsw f...
8177,omg @mention It's not a rumor: Apple is openin...,Apple,1.0,Apple,omg rumor apple opening temporary store downt...
8102,"Line for the iPad 2, at the Apple Store. No de...",iPad,1.0,Apple,line ipad apple store demo units pm sxsw f...


In [10]:
from sklearn.model_selection import train_test_split
#creating new df where sentiment is either positive or negative
bi_tar = df[(df['Sentiment'] == 0)| (df['Sentiment'] == 1)]

X = bi_tar['processed_text']
y = bi_tar['Sentiment']

X_train_bi, X_test_bi, y_train_bi, y_test_bi = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
multi_tar = df.copy()
X = multi_tar['processed_text']
y = multi_tar['Sentiment']

y_dummies = pd.get_dummies(y)
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X, y_dummies, test_size=0.2, random_state=42)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

def csr_Tfid_vect(X_train,X_test):
    vectorizer = TfidfVectorizer()
    tf_idf_train = vectorizer.fit_transform(X_train)
    tf_idf_test = vectorizer.transform(X_test)

    tf_idf_train = csr_matrix(tf_idf_train)
    tf_idf_test = csr_matrix(tf_idf_test)

    return tf_idf_train,tf_idf_test

X_tf_idf_train_bi,X_tf_idf_test_bi = csr_Tfid_vect(X_train_bi,X_test_bi)

In [13]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.preprocessing import text, sequence

def token_seq(feature):
    tokenizer = text.Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(list(feature))
    list_tokenized = tokenizer.texts_to_sequences(feature)
    seq = sequence.pad_sequences(list_tokenized, maxlen=100)
    return seq

## Text Analysis

In [14]:
from  nltk import FreqDist
import string

big_sentence = ' '.join(df['tweet_text'])
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tweets_raw = nltk.regexp_tokenize(big_sentence, pattern)
tweets_raw = [word.lower() for word in tweets_raw]
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
tweets_raw_stopped = [word for word in tweets_raw if word not in stopwords_list]
tweets_freqdist = FreqDist(tweets_raw_stopped)
total_word_count = sum(tweets_freqdist.values())
tweets_freqdist_top_10 = tweets_freqdist.most_common(10)
print(f'{"Word":10} Normalized Frequency')
for word in tweets_freqdist_top_10:
    normalized_frequency = word[1] / total_word_count
    print(f'{word[0]:10} {normalized_frequency:^20.4}')

Word       Normalized Frequency
sxsw              0.0858       
mention          0.06442       
link             0.03821       
rt               0.02743       
ipad             0.02671       
google            0.022        
apple            0.01969       
quot             0.01503       
iphone           0.01414       
store            0.01316       


In [15]:
from nltk.collocations import BigramCollocationFinder

bigram_measures = nltk.collocations.BigramAssocMeasures()
tweets_finder = BigramCollocationFinder.from_words(tweets_raw_stopped)
tweets_scored = tweets_finder.score_ngrams(bigram_measures.raw_freq)
tweets_scored[:15]

[(('rt', 'mention'), 0.02666802617674867),
 (('sxsw', 'link'), 0.008527835968929014),
 (('link', 'sxsw'), 0.007656513598190615),
 (('sxsw', 'rt'), 0.006275374946701025),
 (('mention', 'mention'), 0.005728481118258838),
 (('mention', 'sxsw'), 0.005478207671344618),
 (('apple', 'store'), 0.005348436254426132),
 (('sxsw', 'mention'), 0.004755195491370201),
 (('link', 'rt'), 0.004718117943679205),
 (('mention', 'google'), 0.004356611853691997),
 (('social', 'network'), 0.0040970690198550265),
 (('new', 'social'), 0.003781909864481563),
 (('mention', 'rt'), 0.0031886691014256317),
 (('network', 'called'), 0.003021820136816151),
 (('store', 'sxsw'), 0.003021820136816151)]

SXSW is best known for its conference and festivals that celebrate the convergence of tech, film, music, education, and culture.
RT is the first Russian 24/7 English-language news channel which brings the Russian view on global news.

In [16]:
from gensim.models import Word2Vec
from nltk import word_tokenize

data = df['processed_text'].map(word_tokenize)
model = Word2Vec(data, window=5, min_count=1, workers=4)
model.train(data, total_examples=model.corpus_count, epochs=10)

(724523, 978660)

In [17]:
wv = model.wv
wv.most_similar('sxsw')

[('notsosecret', 0.6787000894546509),
 ('samsungsxsw', 0.6743574142456055),
 ('craps', 0.6580419540405273),
 ('socmedia', 0.6464982628822327),
 ('ireport', 0.6461699604988098),
 ('south', 0.6448580026626587),
 ('cabs', 0.6438364386558533),
 ('frickin', 0.6380432844161987),
 ('greatergood', 0.6293945908546448),
 ('ismparty', 0.627713143825531)]

In [18]:
wv.most_similar(negative='sxsw')

[('sqchat', 0.37491530179977417),
 ('delivery', 0.3722917139530182),
 ('ringing', 0.3664971590042114),
 ('newapplestoreaustin', 0.3149334192276001),
 ('warmer', 0.3019205331802368),
 ('forever', 0.27612364292144775),
 ('mocking', 0.26970696449279785),
 ('aug', 0.25624948740005493),
 ('petricone', 0.25289714336395264),
 ('questioner', 0.24092993140220642)]

## Modeling & Evaluation

**Baseline Model**: Sentiment is either positive(1)or negative(0)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([('Random Forest', RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([('Support Vector Machine', SVC())])
lr = Pipeline([('Logistic Regression', LogisticRegression())])

models = [('Random Forest', rf),
          ('Support Vector Machine', svc),
          ('Logistic Regression', lr)]

scores = [(name, cross_val_score(model,X_tf_idf_train_bi, y_train_bi, cv=2).mean()) for name, model, in models]
scores 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   10.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


[('Random Forest', 0.8630540762312359),
 ('Support Vector Machine', 0.8568296515587877),
 ('Logistic Regression', 0.8443789251256308)]

In [20]:
X_t =  token_seq(X_train_bi) 

In [21]:
model_1 = Sequential()

model_1.add(Dense(units=64, input_shape=(100,)))
model_1.add(Dropout(0.5))

model_1.add(Dense(32, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(1, activation='sigmoid'))

model_1.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=["accuracy"]
)

model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                6464      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8577 (33.50 KB)
Trainable params: 8577 (33.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model_1.fit(X_t, y_train_bi, epochs=15, batch_size=32, validation_split=0.1)

Epoch 1/15
77/77 [==============================] - 7s 25ms/step - loss: 162.7058 - accuracy: 0.6736 - val_loss: 73.1151 - val_accuracy: 0.8248
Epoch 2/15
77/77 [==============================] - 1s 13ms/step - loss: 92.1471 - accuracy: 0.6927 - val_loss: 39.7439 - val_accuracy: 0.8212
Epoch 3/15
77/77 [==============================] - 1s 10ms/step - loss: 53.8014 - accuracy: 0.6931 - val_loss: 10.7263 - val_accuracy: 0.7482
Epoch 4/15
77/77 [==============================] - 1s 9ms/step - loss: 33.2763 - accuracy: 0.6911 - val_loss: 4.9072 - val_accuracy: 0.7664
Epoch 5/15
77/77 [==============================] - 1s 9ms/step - loss: 18.7806 - accuracy: 0.7293 - val_loss: 1.8408 - val_accuracy: 0.8175
Epoch 6/15
77/77 [==============================] - 1s 11ms/step - loss: 11.2163 - accuracy: 0.7428 - val_loss: 0.9618 - val_accuracy: 0.6569
Epoch 7/15
77/77 [==============================] - 1s 9ms/step - loss: 6.8843 - accuracy: 0.7676 - val_loss: 0.7659 - val_accuracy: 0.8321
Epoch 

**Iterated Model**: Sentiment is either positive(1),negative(0),No emotion toward brand or product(2) or Not clear(3)

In [23]:
X_t_multi = token_seq(X_train_multi)

In [24]:
model_2 = Sequential()

model_2.add(Dense(64, activation='relu', input_shape=(100,)))
model_2.add(Dropout(0.5))

model_2.add(Dense(3, activation='softmax'))

model_2.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

model_2.fit(X_t_multi, y_train_multi, epochs=15, batch_size=32, validation_split=0.1)

Epoch 1/15
197/197 [==============================] - 5s 11ms/step - loss: 269.9518 - accuracy: 0.4439 - val_loss: 121.5970 - val_accuracy: 0.5573
Epoch 2/15
197/197 [==============================] - 2s 10ms/step - loss: 100.9804 - accuracy: 0.4665 - val_loss: 50.4115 - val_accuracy: 0.5315
Epoch 3/15
197/197 [==============================] - 2s 9ms/step - loss: 42.6108 - accuracy: 0.4689 - val_loss: 24.6269 - val_accuracy: 0.5372
Epoch 4/15
197/197 [==============================] - 2s 8ms/step - loss: 19.2499 - accuracy: 0.4825 - val_loss: 12.2921 - val_accuracy: 0.5659
Epoch 5/15
197/197 [==============================] - 1s 7ms/step - loss: 8.8626 - accuracy: 0.4928 - val_loss: 6.7513 - val_accuracy: 0.5616
Epoch 6/15
197/197 [==============================] - 2s 8ms/step - loss: 5.1705 - accuracy: 0.4995 - val_loss: 4.0073 - val_accuracy: 0.5473
Epoch 7/15
197/197 [==============================] - 1s 7ms/step - loss: 3.0224 - accuracy: 0.5266 - val_loss: 2.7932 - val_accuracy: 

In [25]:
total_vocabulary = set(word for headline in data for word in headline)
print('There are {} unique tokens in the dataset.'.format(len(total_vocabulary)))

There are 9174 unique tokens in the dataset.


In [26]:
import numpy as np
glove = {}
with open('glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [27]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # Takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # it can't be used in a scikit-learn pipeline  
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [28]:
c = [d for d in df['Sentiment']]

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Random Forest', RandomForestClassifier(n_estimators=100, verbose=True))])

models = [('Random Forest', rf)]

scores = [(name, cross_val_score(model, data,c, cv=2).mean()) for name, model, in models]
scores 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   22.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   22.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


[('Random Forest', 0.642701835914244)]

In [30]:
rf.fit(data,c)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   47.1s finished


Pipeline(steps=[('Word2Vec Vectorizer',
                 <__main__.W2vVectorizer object at 0x000001D612E335B0>),
                ('Random Forest', RandomForestClassifier(verbose=True))])

In [31]:
def map_to_new_column(value):
    if value == 0.0:
        return 'negative'
    elif value == 1.0:
        return 'positive'
    elif value == 2.0:
        return 'neutral'
    else:
        return 'unknown'  

In [32]:
def sentiment_predict(filepath,tweet_column,dir_at,model):
    with open(filepath, 'r', encoding='utf-8') as file:
        cleaned_text = remove_non_utf8(file.read())
    df = pd.read_csv(io.StringIO(cleaned_text))

    df[tweet_column].fillna('N/A', inplace=True)
    df['processed_text'] = df[tweet_column].map(clean_and_preprocess_text)

    df[dir_at].fillna('N/A', inplace=True)
    df['brand'] = df[dir_at].apply(assign_brand)

    data = df['processed_text'].map(word_tokenize)
    pred = model.predict(data)

    df['pred'] = pred
    df['prediction'] = df['pred'].apply(map_to_new_column)
    
    return df


In [33]:
filepath = 'data/judge_1377884607_tweet_product_company.csv'
tweet_column = 'tweet_text'
dir_at = 'emotion_in_tweet_is_directed_at'
model = rf

pred_df = sentiment_predict(filepath,tweet_column,dir_at,model)
pred_df.sample(5)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,processed_text,brand,pred,prediction
7951,Free Smart Recorder for this weekend only {lin...,iPhone,Positive emotion,free smart recorder weekend link via iphone ...,Apple,1.0,positive
7162,Does the pop up Apple store in Austin still ha...,N/A,No emotion toward brand or product,pop apple store austin still ipad stock lazy...,N/A,2.0,neutral
5199,RT @mention Apple is opening a store at #SXSW....,N/A,No emotion toward brand or product,rt apple opening store sxsw link,N/A,2.0,neutral
25,RT @LaurieShook: I'm looking forward to the #S...,iPad,Positive emotion,rt im looking forward smcdallas pre sxsw part...,Apple,1.0,positive
5670,RT @mention Google to Launch Major New Social ...,N/A,Positive emotion,rt google launch major new social network call...,N/A,2.0,neutral


In [34]:
#positive apple tweets
apple_pos = pred_df[(pred_df['brand'] == 'Apple') & (pred_df['prediction'] == 'positive')]
#negative apple tweets
apple_neg = pred_df[(pred_df['brand'] == 'Apple') & (pred_df['prediction'] == 'negative')]
apple_neg.emotion_in_tweet_is_directed_at.value_counts()

iPad                              116
iPhone                             98
Apple                              91
iPad or iPhone App                 60
Other Apple product or service      2
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [35]:
#positive Google tweets
Google_pos = pred_df[(pred_df['brand'] == 'Google') & (pred_df['prediction'] == 'positive')]
#negative google tweets
Google_neg = pred_df[(pred_df['brand'] == 'Google') & (pred_df['prediction'] == 'negative')]
Google_neg.emotion_in_tweet_is_directed_at.value_counts()

Google                             61
Other Google product or service    46
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [36]:
#positive Android tweets
Android_pos = pred_df[(pred_df['brand'] == 'Android') & (pred_df['prediction'] == 'positive')]
#negative google tweets
Android_neg = pred_df[(pred_df['brand'] == 'Android') & (pred_df['prediction'] == 'negative')]
Android_neg.emotion_in_tweet_is_directed_at.value_counts()

Android        7
Android App    7
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [37]:
import pickle

# with open('model_rf.pkl', 'wb') as file:
#     pickle.dump(rf, file)

# file.close()

## Deployment